Update following to connect to your own database

In [1]:
dbconfig = {
    "host":"tascloud",
    "port":3306,
    "user":"test",
    "password":"1-TestPassword",
    "database":"little_lemon_db"
}


In [2]:
from mysql.connector import pooling
try:
    cpool = pooling.MySQLConnectionPool(pool_name="pynative_pool",
                                                  pool_size=2,
                                                  pool_reset_session=True,
                                                  **dbconfig)
except Exception as e:
    print("Could not create a connection pool. See error for details")
    print(e.args)

In [3]:
#Procedures

proc_peak_hours = """
CREATE PROCEDURE PeakHours()
SELECT HOUR(BookingSlot) AS BookingHour, COUNT(*) AS BookingCount
FROM little_lemon_db.Bookings 
GROUP BY HOUR(BookingSlot)
ORDER BY BookingCount DESC
"""

proc_guest_status = """
CREATE PROCEDURE GuestStatus()
SELECT
	concat(B.GuestFirstName, ' ', B.GuestLastName) AS GuestName, 
    CASE 
		WHEN E.Role = 'Manager' OR E.Role = 'Assistant Manager' THEN 'Ready to pay'
        WHEN E.Role = 'Head Chef' THEN 'Ready to serve'
        WHEN E.Role = 'Assistant Chef' THEN 'Preparing Order'
        WHEN E.Role = 'Head Waiter' THEN 'Order served'
	END AS Status
FROM Bookings AS B
LEFT JOIN Employees AS E
	ON B.EmployeeID = E.EmployeeID"""




In [4]:
# Creating procedures
# Peak Hours
con1 = cpool.get_connection()
cmd1 = con1.cursor()
try:
    cmd1.execute(proc_peak_hours)
except Exception as err:
    print(err, "PeakHours")

# Guest Statuses
con2 = cpool.get_connection()
cmd2 = con2.cursor()
try:
    cmd2.execute(proc_guest_status)
except Exception as err:
    print(err, "GuestStatus")

1304 (42000): PROCEDURE PeakHours already exists


In [5]:
# Running the proc
cmd1.callproc("PeakHours")
results = next( cmd1.stored_results() )
dataset = results.fetchall()
column_names = []
#getting column names
for result in results.description:
    column_names.append(result[0])
print(column_names)
for data in dataset:
    print(data)
con1.close()

['BookingHour', 'BookingCount']
(19, 2)
(15, 1)
(17, 1)
(18, 1)
(20, 1)


In [6]:
# Running the proc
cmd2.callproc("GuestStatus")
results = next( cmd2.stored_results() )
dataset = results.fetchall()
column_names = []
#getting column names
for result in results.description:
    column_names.append(result[0])
print(column_names)
for data in dataset:
    print(data)
con2.close()

['GuestName', 'Status']
('Anna Iversen', 'Ready to pay')
('Joakim Iversen', 'Ready to pay')
('Vanessa McCarthy', 'Ready to serve')
('Marcos Romero', 'Preparing Order')
('Hiroki Yamane', 'Ready to pay')
('Diana Pinto', 'Order served')
